<a href="https://colab.research.google.com/github/DonghaeSuh/NLP_tensorflow2/blob/main/6_ChatBot/Transformer_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 작업 공간 접근

In [2]:
ls

drive/  sample_data/


In [3]:
cd drive/MyDrive/ChatBot_Transformer/

/content/drive/MyDrive/ChatBot_Transformer


In [4]:
ls

data_in/          preprocess.py        __pycache__/
Preprocess.ipynb  preprocess_py.ipynb  Transformer_ChatBot.ipynb


### Konlpy 설치

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

# 모듈 불러오기

In [ ]:
import tensorflow as tf
import numpy as np

from konlpy.tag import Twitter
import pandas as pd
import tensorflow as tf
import enum
import os
import re
import json
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

from preprocess import *

## TEST

### mask

In [ ]:
import tensorflow as tf

In [ ]:
b=1-tf.linalg.band_part(tf.ones((2,2)),-1,0)

In [ ]:
b*-1e9

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-0.e+00, -1.e+09],
       [-0.e+00, -0.e+00]], dtype=float32)>

In [ ]:
import numpy as np

In [ ]:
a=np.array([[1,2],[3,4]])

In [ ]:
a+=b*-1e9

In [ ]:
a

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 1.e+00, -1.e+09],
       [ 3.e+00,  4.e+00]], dtype=float32)>

### tf.matamul

In [ ]:
a=np.array([[[[1,2,3],[3,4,5]],[[5,6,7],[7,8,9]]],[[[1,2,3],[3,4,5]],[[5,6,7],[7,8,9]]]])

In [ ]:
a.shape

(2, 2, 2, 3)

In [ ]:
tf.matmul(a,a,transpose_b=True)

<tf.Tensor: shape=(2, 2, 2, 2), dtype=int64, numpy=
array([[[[ 14,  26],
         [ 26,  50]],

        [[110, 146],
         [146, 194]]],


       [[[ 14,  26],
         [ 26,  50]],

        [[110, 146],
         [146, 194]]]])>

### tf.reshape & tf.transpose  (4차원)

In [ ]:
a = np.array([[[1,2,3,4],[4,5,6,7]],[[7,8,9,10],[10,11,12,13]]])
a

array([[[ 1,  2,  3,  4],
        [ 4,  5,  6,  7]],

       [[ 7,  8,  9, 10],
        [10, 11, 12, 13]]])

In [ ]:
b=tf.reshape(a,(2,-1,2,2))
b

<tf.Tensor: shape=(2, 2, 2, 2), dtype=int64, numpy=
array([[[[ 1,  2],
         [ 3,  4]],

        [[ 4,  5],
         [ 6,  7]]],


       [[[ 7,  8],
         [ 9, 10]],

        [[10, 11],
         [12, 13]]]])>

In [ ]:
tf.transpose(b) # perm = [3,2,1,0]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=int64, numpy=
array([[[[ 1,  7],
         [ 4, 10]],

        [[ 3,  9],
         [ 6, 12]]],


       [[[ 2,  8],
         [ 5, 11]],

        [[ 4, 10],
         [ 7, 13]]]])>

In [ ]:
tf.transpose(b,[0,2,1,3])

<tf.Tensor: shape=(2, 2, 2, 2), dtype=int64, numpy=
array([[[[ 1,  2],
         [ 4,  5]],

        [[ 3,  4],
         [ 6,  7]]],


       [[[ 7,  8],
         [10, 11]],

        [[ 9, 10],
         [12, 13]]]])>

In [ ]:
tf.transpose(b,[3,1,2,0])

<tf.Tensor: shape=(2, 2, 2, 2), dtype=int64, numpy=
array([[[[ 1,  7],
         [ 3,  9]],

        [[ 4, 10],
         [ 6, 12]]],


       [[[ 2,  8],
         [ 4, 10]],

        [[ 5, 11],
         [ 7, 13]]]])>

In [ ]:
tf.reshape(a,(2,2,-1,2))

<tf.Tensor: shape=(2, 2, 2, 2), dtype=int64, numpy=
array([[[[ 1,  2],
         [ 3,  4]],

        [[ 4,  5],
         [ 6,  7]]],


       [[[ 7,  8],
         [ 9, 10]],

        [[10, 11],
         [12, 13]]]])>

In [ ]:
x = tf.constant([[[ 1,  2,  3],
                  [ 4,  5,  6]],
                 [[ 7,  8,  9],
                  [10, 11, 12]]])

In [ ]:
x

<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]]], dtype=int32)>

In [ ]:
tf.transpose(x) #perm=[2,1,0]

<tf.Tensor: shape=(3, 2, 2), dtype=int32, numpy=
array([[[ 1,  7],
        [ 4, 10]],

       [[ 2,  8],
        [ 5, 11]],

       [[ 3,  9],
        [ 6, 12]]], dtype=int32)>

In [ ]:
tf.transpose(x,perm=[1,2,0])

<tf.Tensor: shape=(2, 3, 2), dtype=int32, numpy=
array([[[ 1,  7],
        [ 2,  8],
        [ 3,  9]],

       [[ 4, 10],
        [ 5, 11],
        [ 6, 12]]], dtype=int32)>

In [ ]:
tf.transpose(x,perm=[2,0,1])

<tf.Tensor: shape=(3, 2, 2), dtype=int32, numpy=
array([[[ 1,  4],
        [ 7, 10]],

       [[ 2,  5],
        [ 8, 11]],

       [[ 3,  6],
        [ 9, 12]]], dtype=int32)>

In [ ]:
a=np.array([[1,2,3],[4,5,6]])
a

array([[1, 2, 3],
       [4, 5, 6]])

In [ ]:
tf.transpose(a) # perm = [1,0]

<tf.Tensor: shape=(3, 2), dtype=int64, numpy=
array([[1, 4],
       [2, 5],
       [3, 6]])>

perm의 해당 차원(axis)에 따른 연산 횟수(이어붙이기)가, 새로 만들어지는 행렬의 행 차원이 될 것이므로, 현재 차원이 perm의 위치에 따라 적절하게 변하다.

perm의 마지막 axis부터 차례대로 뒷 차원부터 쌓여서 결과가 나올텐데,
각 차원에 해당하는 이어붙이기 개수가 그 차워의 개수가 될 것이므로,\
차원이 우리가 원하는 대로 변형된다.

### list의 [...]

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
a=np.array([[1,2,3],[4,5,6]])
a

array([[1, 2, 3],
       [4, 5, 6]])

In [ ]:
a[...]

array([[1, 2, 3],
       [4, 5, 6]])

In [ ]:
a[np.newaxis,...]

array([[[1, 2, 3],
        [4, 5, 6]]])

In [ ]:
a[...,np.newaxis]

array([[[1],
        [2],
        [3]],

       [[4],
        [5],
        [6]]])

### BroadCastiing numpy

In [ ]:
a=np.array([[[1,2,3],[4,5,6]]])
b=np.array([[[1,2,3],[4,5,6]],[[7,8,9],[10,11,12]]])

b+a

array([[[ 2,  4,  6],
        [ 8, 10, 12]],

       [[ 8, 10, 12],
        [14, 16, 18]]])

In [ ]:
a=tf.cast(np.array([[False,True,True],[False,False,True]]),tf.float32) # 2 x 3
c=a[:,None,None,:] # 2 x 1 x 1 x 3

In [ ]:
c

<tf.Tensor: shape=(2, 1, 1, 3), dtype=float32, numpy=
array([[[[0., 1., 1.]]],


       [[[0., 0., 1.]]]], dtype=float32)>

In [ ]:
b=np.array([[0,1,1],[0,0,1],[0,0,0]])
tf.maximum(c,b)

<tf.Tensor: shape=(2, 1, 3, 3), dtype=float32, numpy=
array([[[[0., 1., 1.],
         [0., 1., 1.],
         [0., 1., 1.]]],


       [[[0., 1., 1.],
         [0., 0., 1.],
         [0., 0., 1.]]]], dtype=float32)>